## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dakar,SN,14.6937,-17.4441,86.13,74,75,13.80,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,42.80,81,100,14.97,overcast clouds
2,2,Dikson,RU,73.5069,80.5464,42.55,95,99,9.86,overcast clouds
3,3,Avera,US,33.1940,-82.5271,82.45,64,100,4.21,overcast clouds
4,4,Mar Del Plata,AR,-38.0023,-57.5575,54.09,84,100,13.80,overcast clouds


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dakar,SN,14.6937,-17.4441,86.13,74,75,13.80,broken clouds
3,3,Avera,US,33.1940,-82.5271,82.45,64,100,4.21,overcast clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,75.16,91,95,5.01,overcast clouds
6,6,Ponta Do Sol,PT,32.6667,-17.1000,88.43,64,100,3.98,overcast clouds
7,7,Margate,GB,51.3813,1.3862,73.87,68,94,4.00,overcast clouds
8,8,Kavieng,PG,-2.5744,150.7967,80.65,77,41,10.18,light rain
10,10,Halifax,CA,44.6453,-63.5724,85.89,60,1,11.50,clear sky
11,11,Eldorado,BR,-23.7869,-54.2836,71.56,58,2,7.27,clear sky
12,12,Harper,LR,4.3750,-7.7169,77.88,75,41,9.24,scattered clouds
15,15,Sao Joao Da Barra,BR,-21.6403,-41.0511,73.49,60,93,13.15,overcast clouds


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                356
City                   356
Country                356
Lat                    356
Lng                    356
Max Temp               356
Humidity               356
Cloudiness             356
Wind Speed             356
Current Description    356
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dakar,SN,14.6937,-17.4441,86.13,74,75,13.80,broken clouds
3,3,Avera,US,33.1940,-82.5271,82.45,64,100,4.21,overcast clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,75.16,91,95,5.01,overcast clouds
6,6,Ponta Do Sol,PT,32.6667,-17.1000,88.43,64,100,3.98,overcast clouds
7,7,Margate,GB,51.3813,1.3862,73.87,68,94,4.00,overcast clouds


In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dakar,SN,86.13,broken clouds,14.6937,-17.4441,
3,Avera,US,82.45,overcast clouds,33.1940,-82.5271,
5,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
6,Ponta Do Sol,PT,88.43,overcast clouds,32.6667,-17.1000,
7,Margate,GB,73.87,overcast clouds,51.3813,1.3862,
8,Kavieng,PG,80.65,light rain,-2.5744,150.7967,
10,Halifax,CA,85.89,clear sky,44.6453,-63.5724,
11,Eldorado,BR,71.56,clear sky,-23.7869,-54.2836,
12,Harper,LR,77.88,scattered clouds,4.3750,-7.7169,
15,Sao Joao Da Barra,BR,73.49,overcast clouds,-21.6403,-41.0511,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]  
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dakar,SN,86.13,broken clouds,14.6937,-17.4441,Novotel Dakar
3,Avera,US,82.45,overcast clouds,33.1940,-82.5271,
5,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Ponta Do Sol,PT,88.43,overcast clouds,32.6667,-17.1000,Hotel do Campo
7,Margate,GB,73.87,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
...,...,...,...,...,...,...,...
679,Cavalcante,BR,83.62,scattered clouds,-13.7975,-47.4583,Pousada Vila dos Ipês
683,Kerema,PG,74.97,overcast clouds,-7.9631,145.7785,Frena Lodge
685,Tapachula,MX,86.52,broken clouds,14.9000,-92.2833,Hotel Casa Mexicana
687,Bandiagara,ML,87.96,broken clouds,14.3501,-3.6104,Hotel de la Falaise


In [44]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df=hotel_df.replace('', np.NaN)

In [46]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dakar,SN,86.13,broken clouds,14.6937,-17.4441,Novotel Dakar
3,Avera,US,82.45,overcast clouds,33.1940,-82.5271,NaN
5,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Ponta Do Sol,PT,88.43,overcast clouds,32.6667,-17.1000,Hotel do Campo
7,Margate,GB,73.87,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
8,Kavieng,PG,80.65,light rain,-2.5744,150.7967,Nusa Island Retreat
10,Halifax,CA,85.89,clear sky,44.6453,-63.5724,Hotel Halifax
11,Eldorado,BR,71.56,clear sky,-23.7869,-54.2836,Villa Verde Hotel
12,Harper,LR,77.88,scattered clouds,4.3750,-7.7169,Screensaver
15,Sao Joao Da Barra,BR,73.49,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas


In [47]:
clean_hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dakar,SN,86.13,broken clouds,14.6937,-17.4441,Novotel Dakar
5,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Ponta Do Sol,PT,88.43,overcast clouds,32.6667,-17.1000,Hotel do Campo
7,Margate,GB,73.87,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
8,Kavieng,PG,80.65,light rain,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
679,Cavalcante,BR,83.62,scattered clouds,-13.7975,-47.4583,Pousada Vila dos Ipês
683,Kerema,PG,74.97,overcast clouds,-7.9631,145.7785,Frena Lodge
685,Tapachula,MX,86.52,broken clouds,14.9000,-92.2833,Hotel Casa Mexicana
687,Bandiagara,ML,87.96,broken clouds,14.3501,-3.6104,Hotel de la Falaise


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "City_Data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</d><dd>{Hotel Name}</dd>
<dt>City</d><dd>{City}</dd>
<dt>Country</d><dd>{Country}</dd>
<dt>Weather</d><dd>{Current Description}</dd>
<dt>Max Temp</d><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))